In [42]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors

In [43]:
df = pd.read_csv('dataset.csv')

In [44]:
class Recommender:
    
    def __init__(self):
        self.df = pd.read_csv('dataset.csv')
    
    def get_features(self):
        #getting dummies of dataset
        nutrient_dummies = self.df.Nutrient.str.get_dummies()
        disease_dummies = self.df.Disease.str.get_dummies(sep=' ')
        diet_dummies = self.df.Diet.str.get_dummies(sep=' ')
        feature_df = pd.concat([nutrient_dummies,disease_dummies,diet_dummies],axis=1)
     
        return feature_df
    
    def k_neighbor(self,inputs):
        
        feature_df = self.get_features()
        
        #initializing model with k=20 neighbors
        model = NearestNeighbors(n_neighbors=40,algorithm='ball_tree')
        
        # fitting model with dataset features
        model.fit(feature_df)
        
        df_results = pd.DataFrame(columns=list(self.df.columns))
        
      
        # getting distance and indices for k nearest neighbor
        distnaces , indices = model.kneighbors(inputs)

        for i in list(indices):
            df_results = df_results._append(self.df.loc[i])
                
        df_results = df_results.filter(['Name','Nutrient','Veg_Non','Price','Review','Diet','Disease','description'])
        df_results = df_results.drop_duplicates(subset=['Name'])
        df_results = df_results.reset_index(drop=True)
        return df_results

In [45]:
ob = Recommender()
data = ob.get_features()

total_features = data.columns
d = dict()
for i in total_features:
    d[i]= 0
print(d)

{'calcium': 0, 'carbohydrates': 0, 'chloride': 0, 'fiber': 0, 'iodine': 0, 'iron': 0, 'magnesium': 0, 'manganese': 0, 'phosphorus': 0, 'potassium': 0, 'protien': 0, 'selenium': 0, 'sodium': 0, 'vitamin_a': 0, 'vitamin_c': 0, 'vitamin_d': 0, 'vitamin_e': 0, 'anemia': 0, 'cancer': 0, 'diabeties': 0, 'eye_disease': 0, 'goitre': 0, 'heart_disease': 0, 'hypertension': 0, 'kidney_disease': 0, 'obesity': 0, 'pregnancy': 0, 'rickets': 0, 'scurvy': 0, 'Mediterranean_diet': 0, 'alkaline_diet': 0, 'dash_diet': 0, 'gluten_free_diet': 0, 'high_fiber_diet': 0, 'high_protien_diet': 0, 'hormone_diet': 0, 'ketogenic_diet': 0, 'low_carb_diet': 0, 'low_fat_diet': 0, 'low_sodium_diet': 0, 'omni_diet': 0, 'paleo_diet': 0, 'type_a_diet': 0, 'type_o_diet': 0, 'vegan_diet': 0}


In [46]:
sample_input = ['high_protien_diet','gluten_free_diet','diabeties','anemia','calcium','protien']

for i in sample_input:
    
    d[i] = 1

final_input = list(d.values())

In [47]:
results = ob.k_neighbor([final_input]) # pass 2d array []
results

,Name,Nutrient,Veg_Non,Price,Diet,Disease,description
0,corn pulao,protien,veg,410,high_protien_diet low_fat_diet gluten_free_diet,diabeties goitre,"बासमती चावल, अमेरिकन कॉर्न के दाने, जैतून का त..."
1,veg summer rolls,carbohydrates,veg,545,high_protien_diet,diabeties,"rice paper sheets, iceberg lettuce, carrot, be..."
2,oats &amp; shallots pulao,protien,veg,470,high_protien_diet dash_diet gluten_free_diet,hypertension,"चावल, हरा धनिया, हरी मिर्च, प्याज , दालचीनी, इ..."
3,oats khichdi,protien,veg,690,high_protien_diet dash_diet gluten_free_diet,hypertension,"क्विक कुकिंग ओट्स, मूंगदाल, जीरा, हल्दी पाउडर,..."
4,almond pearls,protien,veg,550,gluten_free_diet,hypertension,"toasted almonds, blueberries, oats, corn flake..."
5,apple kheer,vitamin_e,veg,355,high_protien_diet,obesity diabeties,"apples, basmati rice, nuscovado sugar (you can..."
6,cajun spiced turkey wrapped with bacon,vitamin_c,non-veg,485,high_protien_diet,cancer,"turkey breast, cajun spice, spinach leaves (co..."
7,cinnamon star cookies,protien,veg,530,ketogenic_diet,cancer diabeties hypertension,"मक्खन, कैस्टर शुगर, क्रिसमस मिक्स स्पाइसी, दाल..."
8,apple rabdi,calcium,veg,335,high_protien_diet vegan_diet low_fat_diet ket...,obesity diabeties,"apples, milk, sugar, green cardamoms, almonds ..."
9,lemon honey glazed sous vide corn on the cob,protien,non-veg,310,low_fat_diet gluten_free_diet high_fiber_diet...,diabeties goitre scurvy,"young corn on the cob, honey, lemon juice, gar..."


In [48]:
my_df_check=pd.read_csv('dataset.csv')
my_df_check.head()

,Meal_Id,Name,catagory,description,Veg_Non,Nutrient,Disease,Diet,Price
0,meal_id1,summer squash salad,salad,"white balsamic vinegar, lemon juice, lemon rin...",veg,fiber,obesity diabeties hypertension goitre,alkaline_diet low_fat_diet ketogenic_diet low...,485
1,meal_id2,chicken minced salad,salad,"olive oil, chicken mince, garlic (minced), oni...",non-veg,fiber,anemia pregnancy hypertension rickets goitre ...,low_fat_diet low_carb_diet ketogenic_diet low...,600
2,meal_id3,sweet chilli almonds,chilli,"almonds whole, egg white, curry leaves, salt, ...",veg,vitamin_a,hypertension scurvy heart_disease goitre kidn...,alkaline_diet low_fat_diet paleo_diet Mediter...,255
3,meal_id4,tricolour salad,salad,"vinegar, honey/sugar, soy sauce, salt, garlic ...",veg,fiber,obesity goitre hypertension,low_fat_diet ketogenic_diet low_sodium_diet h...,615
4,meal_id5,gluten-free christmas cake,cake,"christmas dry fruits (pre-soaked), orange zest...",veg,vitamin_a,goitre kidney_disease,high_protien_diet,465


In [49]:
my_df_check.Diet.unique()

array([' alkaline_diet low_fat_diet ketogenic_diet low_sodium_diet high_fiber_diet high_protien_diet dash_diet',
       ' low_fat_diet low_carb_diet ketogenic_diet low_sodium_diet vegan_diet high_fiber_diet hormone_diet high_protien_diet type_a_diet',
       ' alkaline_diet low_fat_diet paleo_diet Mediterranean_diet low_sodium_diet high_fiber_diet high_protien_diet dash_diet',
       ' low_fat_diet ketogenic_diet low_sodium_diet high_fiber_diet high_protien_diet',
       ' high_protien_diet',
       ' high_protien_diet vegan_diet low_fat_diet ketogenic_diet',
       ' high_protien_diet high_fiber_diet ketogenic_diet',
       ' alkaline_diet',
       ' alkaline_diet low_fat_diet ketogenic_diet gluten_free_diet vegan_diet high_protien_diet',
       ' alkaline_diet ketogenic_diet gluten_free_diet vegan_diet high_protien_diet dash_diet',
       ' alkaline_diet low_fat_diet paleo_diet ketogenic_diet low_sodium_diet gluten_free_diet vegan_diet high_fiber_diet high_protien_diet dash_diet',
  

In [50]:
my_df_check.Nutrient.unique()

array(['fiber', 'vitamin_a', 'calcium', 'magnesium', 'sodium',
       'vitamin_c', 'protien', 'vitamin_e', 'iron', 'selenium',
       'carbohydrates', 'chloride', 'potassium', 'vitamin_d', 'manganese',
       'phosphorus', 'iodine'], dtype=object)

In [51]:
my_df_check.Disease.unique()

array([' obesity diabeties hypertension goitre',
       ' anemia pregnancy hypertension rickets goitre kidney_disease obesity',
       ' hypertension scurvy heart_disease goitre kidney_disease pregnancy',
       ' obesity goitre hypertension', ' goitre kidney_disease',
       ' goitre', ' hypertension heart_disease',
       ' pregnancy scurvy goitre kidney_disease obesity',
       ' obesity cancer hypertension',
       ' pregnancy hypertension scurvy obesity diabeties',
       ' pregnancy hypertension scurvy goitre obesity',
       ' diabeties goitre scurvy', ' scurvy',
       ' anemia cancer heart_disease kidney_disease obesity diabeties',
       ' hypertension',
       ' anemia hypertension cancer scurvy heart_disease kidney_disease diabeties',
       ' goitre hypertension kidney_disease', ' goitre hypertension',
       ' diabeties goitre', ' obesity diabeties',
       ' anemia hypertension scurvy heart_disease goitre kidney_disease pregnancy diabeties',
       ' obesity scurvy hyper

In [52]:
import pandas as pd 
import numpy as np
import warnings
import time
import seaborn as sns
import matplotlib.pyplot as plt
import random
import nltk
from nltk.corpus import stopwords

warnings.filterwarnings('ignore')

In [53]:
class Profile:
    
    df = pd.read_csv('dataset.csv') # static variable
    
    def __init__(self,diet,disease,Nutrient,food_type,favorite_food):
        self.diet = diet
        self.disease = disease
        self.nutrient = Nutrient
        self.type = food_type
        self.like = favorite_food
        self.df2 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df3 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df4 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df5 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df6 = pd.DataFrame(columns=list(Profile.df.columns))
        
    def removestop(self,tokens):
        stop = set(stopwords.words('english'))
        file = open('stopwords.txt','r')
        l = list(file.read().split())
        stop = list(stop) +l
        l = [token for token in tokens if token not in stop]
        return l
            
    def inputs(self,diet,disease,Nutrient,food_type,favorite_food):
        
        if Nutrient:
            self.df2 = Profile.df[Profile.df.Nutrient.isin(Nutrient)]
            self.df2 = self.df2.reset_index()
       
        if food_type:
            self.df2 = self.df2[self.df2.Veg_Non.isin(food_type)]
            self.df2 = self.df2.reset_index()
      
        if diet:
            for i in range(self.df2.shape[0]):
                l = str(self.df2.loc[i,'Diet']).split()
                
                for d in diet:
                    if d in l:
                        self.df4=self.df4._append(self.df2.loc[i])
 
        if disease:
            for i in range(self.df2.shape[0]):
                l = str(self.df2.loc[i,'Disease']).split()
                for d in disease:
                    if d in l:
                        self.df5=self.df5._append(self.df2.loc[i])

        if favorite_food:
            f = self.removestop(favorite_food.split())
            for i in range(Profile.df.shape[0]):
                n = [j.lower() for j in str(Profile.df.loc[i,'Name']).split()]
                for j in n:
                    for k in f:
                        if k==j:
                            self.df6=self.df6._append(Profile.df.loc[i])
            for i in range(Profile.df.shape[0]):
                n = [j.lower() for j in str(Profile.df.loc[i,'description']).split()]
                for j in n:
                    for k in f:
                        if k==j:
                            self.df6=self.df6._append(df.loc[i])
            for i in range(Profile.df.shape[0]):
                n = [j.lower() for j in str(Profile.df.loc[i,'catagory']).split()]
                for j in n:
                    for k in f:
                        if k==j:
                            self.df6=self.df6._append(Profile.df.loc[i])
            
        return self.df2,self.df3,self.df4,self.df5,self.df6
    
    def get_profile(self):
        df2,df3,df4,df5,df6 = self.inputs(self.diet,self.disease,self.nutrient,self.type,self.like)
        
        df_merge = pd.concat([df2,df3,df4,df5,df6],axis=0).drop_duplicates(subset='Name')
        df_merge = df_merge.filter(['Name','Nutrient','Veg_Non','Price','Review','description'])
        print(df_merge.shape)
        
        return df_merge

In [54]:
ob = Profile(['low_sodium_diet','low_fat_diet'],['diabeties'],
             ['calcium','vitamin_c'],['non-veg'],'i love indian')

profile = ob.get_profile()
profile

(14, 5)


,Name,Nutrient,Veg_Non,Price,description
0,almond and chicken momos (without shell),calcium,non-veg,415,"chicken mince, garlic, carrots, spring onion, ..."
1,chicken parmigiana with tomato sauce,vitamin_c,non-veg,485,"for chicken parmigiana:, chicken breast, egg w..."
2,sous-vide salmon tikka,calcium,non-veg,465,"norwegian salmon, black garlic pickle, butter,..."
4,riceless chicken biryani,vitamin_c,non-veg,360,"malabar peppercorn, cinnamon stick, green card..."
5,chicken roulade,calcium,non-veg,665,"chicken breasts, olives, jalapenos, bell peppe..."
6,cheese chicken kebabs,calcium,non-veg,270,"चिकन थाईज़, लहसुन का पेस्ट, लहसुन का पेस्ट, पी..."
7,cajun spiced turkey wrapped with bacon,vitamin_c,non-veg,555,"turkey breast, cajun spice, spinach leaves (co..."
8,seared salmon in tabasco butter,calcium,non-veg,265,"butter, tabasco, chives, salt, salmon fillet, ..."
9,"risotto lobster with parmesan egg pancake, con...",vitamin_c,non-veg,455,"lobster shell, carrot, leeks, garlic cloves, t..."
10,fish with jamun sauce,vitamin_c,non-veg,560,"jamun, sugar, chilli, garlic cloves (minced), ..."
